In [1]:
# calculate amplitude parameters
import numpy as np
import basic
import analysis as ana
import sys
from matplotlib.pyplot import *
import prjlib
colors = ['r','g','b','c','y','m','k','navy']

In [ ]:
def calc_like_skewnorm(Ainp,dat,odat=0.0):
    from scipy import stats
    like = np.zeros(len(Ainp))
    for i, Ai in enumerate(Ainp):
        mA, vA, sA = stats.skewnorm.fit(dat[i,:])
        like[i] = stats.skewnorm.pdf(odat[i],mA,vA,sA)
        #print(Ai,like[i])
    return like

In [ ]:
def fitfunc(z,p0,sigma): 
    return np.exp(-(z/sigma)**p0)

In [ ]:
def bindep(As,sn,Lmin,Lsp,Lmax,spc,bn1,wi):

    fig, ax = subplots(2)
    
    xlim(As[0],.3)
    ylim(0,1)
    ax[0].axhline(.95,color='k',ls='-.')
    ax[0].grid(True)
    ax[1].axhline(.95,color='k',ls='-.')
    ax[1].grid(True)

    bns = [10,20,30]
    for bi, bn in enumerate(bns):
        mb0 = prjlib.multipole_binning(bn,spc=spc,lmin=Lmin,lmax=Lsp)
        mb1 = None
        if Lsp<2048:
            mb1 = prjlib.multipole_binning(bn1,spc='',lmin=Lsp+1,lmax=Lmax)
        print(bn,mb0.bp[0],mb0.bp[1])
        
        PDF, pDF = {}, {}
        #for pi, (patch,rlmin) in enumerate([('deep56','200'),('boss','300')]):
        for pi, (patch,rlmin) in enumerate([('deep56','200')]):
            Ab, oA = prjlib.quadstats(patch,As,sn,mb0,mb1,wi=wi,rlmin=rlmin)

            like0 = calc_like_skewnorm(As,Ab,odat=oA)
            #like1 = prjlib.calc_like(As,Ab,odat=oA,ddat=0.015)
            Ac, PDF[patch], CDF = prjlib.calc_CDF(like0,As)
            PDF[patch] = PDF[patch]/np.max(PDF[patch])
            #ac, pDF[patch], cDF = prjlib.calc_CDF(like1,As)

            ax[pi].plot(Ac,CDF,colors[bi]+'-',label=str(bn))
            ax[pi].plot(Ac,PDF[patch],colors[bi]+'--')
            ax[pi].errorbar(As,like0/np.max(like0),fmt='o',color=colors[bi])

        #PDF['join'] = PDF['deep56']*PDF['boss']
        #CDF = np.cumsum(PDF['join'])/np.sum(PDF['join'])
        #ax[0].plot(Ac,CDF,colors[bi]+'-',label=str(bn))
        #ax[0].plot(Ac,PDF['join'],colors[bi]+'--')

    legend(loc=0,frameon=False)

In [ ]:
As = np.array([0.,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.3])
#patch = 'deep56'
#patch = 'boss'
for wi in ['lcmb','a0p05','a0p1']:
#for wi in ['a0p05']:
    for Lmin in [10,15,20]:
    #for Lmin in [10]:
        #for Lsp in [200,500,2048]:
        for Lsp in [2048]:
            if Lsp==2048:
                bn1s = ['']
            else:
                bn1s = [10,20]
            for bn1 in bn1s:
                rcParams["figure.figsize"] = (8,18)
                bindep(As,100,Lmin,Lsp,2048,'p2',bn1,wi=wi)
                #savefig('figs/cdfA_diag_'+wi+'_Lmin'+str(Lmin)+'_Lsp'+str(Lsp)+'_bn1'+str(bn1)+'.png')
                clf()